In [4]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
import yaml

# Thêm đường dẫn parent vào sys.path để import được sql_query_generator
# Trong Jupyter notebook, __file__ không khả dụng, nên dùng cách khác
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)  # realestate/
grandparent_dir = os.path.dirname(parent_dir)  # real-estate/
sys.path.append(grandparent_dir)

from sql_query_generator import RealEstateSQLGenerator

# Cấu hình hiển thị
plt.style.use('default')
pd.set_option('display.float_format', '{:,.0f}'.format)
sns.set_palette("husl")

# Cấu hình font tiếng Việt
plt.rcParams['font.family'] = ['DejaVu Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

In [5]:
# ============================================================
# Kết nối PostgreSQL
# ============================================================

# Load credentials
POSTGRES_CONFIG = {}
possible_paths = [
    'postgres_credentials.yaml',
    os.path.join(os.path.dirname(__file__), '..', '..', 'postgres_credentials.yaml')
]

for path in possible_paths:
    if os.path.exists(path):
        with open(path, 'r', encoding='utf-8') as f:
            creds = yaml.safe_load(f)
        POSTGRES_CONFIG = creds.get('postgresql', {})
        break

# Fallback defaults
if not POSTGRES_CONFIG.get('host'):
    POSTGRES_CONFIG = {
        'host': 'localhost',
        'port': 5432,
        'database': 'real_estate_db',
        'user': 'postgres',
        'password': 'postgres123'
    }

# Tạo connection string
postgres_url = f"postgresql://{POSTGRES_CONFIG['user']}:{POSTGRES_CONFIG['password']}@{POSTGRES_CONFIG['host']}:{POSTGRES_CONFIG['port']}/{POSTGRES_CONFIG['database']}"

try:
    engine = create_engine(postgres_url)
    print("✅ Kết nối PostgreSQL thành công")
except Exception as e:
    print(f"❌ Lỗi kết nối PostgreSQL: {e}")
    sys.exit(1)

# Tên bảng
table_name = "real_estate_properties"

NameError: name '__file__' is not defined

In [ ]:
# ============================================================
# Chạy Analytics Queries
# ============================================================

sql_gen = RealEstateSQLGenerator()

# Dictionary các queries
queries = {
    'basic_stats': sql_gen.generate_basic_stats_query(table_name),
    'price_by_city': sql_gen.generate_price_by_city_query(table_name),
    'location_distribution': sql_gen.generate_property_type_distribution_query(table_name),
    'price_ranges': sql_gen.generate_price_ranges_query(table_name),
    'recent_trends': sql_gen.generate_recent_trends_query(table_name)
}

# Chạy queries và lưu kết quả
results = {}
for query_name, query_sql in queries.items():
    try:
        print(f"🔍 Đang chạy query: {query_name}")
        with engine.connect() as conn:
            result = conn.execute(text(query_sql))
            df = pd.DataFrame(result.fetchall(), columns=result.keys())
            results[query_name] = df
        print(f"✅ {query_name}: {len(df)} dòng")
    except Exception as e:
        print(f"❌ Lỗi {query_name}: {e}")
        results[query_name] = pd.DataFrame()

print(f"\n📊 Đã chạy {len(results)} queries thành công")

🔍 Dữ liệu của propertyDetails_propertyId = d437663fbb28a3b1:
  propertyDetails_propertyId  Mức giá Diện tích  \
0           d437663fbb28a3b1  12.3 Tỷ      34m2   
1           d437663fbb28a3b1  12.3 Tỷ      34m2   
2           d437663fbb28a3b1  12.3 Tỷ      34m2   
3           d437663fbb28a3b1  12.3 Tỷ      34m2   
4           d437663fbb28a3b1  12.3 Tỷ      34m2   
5           d437663fbb28a3b1  12.3 Tỷ      34m2   

                                Địa chỉ  \
0  Phường Khương Đình,Thanh Xuân,Hà Nội   
1  Phường Khương Đình,Thanh Xuân,Hà Nội   
2  Phường Khương Đình,Thanh Xuân,Hà Nội   
3  Phường Khương Đình,Thanh Xuân,Hà Nội   
4  Phường Khương Đình,Thanh Xuân,Hà Nội   
5  Phường Khương Đình,Thanh Xuân,Hà Nội   

                                                 url  latitude  longitude  
0  https://nhadat247.com.vn/sieu-pham-mat-pho-bui...  20.99821  105.81379  
1  https://nhadat247.com.vn/sieu-pham-mat-pho-bui...  20.99821  105.81379  
2  https://nhadat247.com.vn/sieu-pham-mat-pho-bui..

In [ ]:
# ============================================================
# 1. BIỂU ĐỒ THỐNG KÊ CƠ BẢN
# ============================================================

if 'basic_stats' in results and not results['basic_stats'].empty:
    basic_stats = results['basic_stats'].iloc[0]

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

    # Tổng số bất động sản
    ax1.bar(['Tổng số BĐS'], [basic_stats['total_properties']], color='skyblue')
    ax1.set_title('Tổng số bất động sản', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Số lượng')
    ax1.grid(axis='y', alpha=0.3)

    # Giá trung bình
    ax2.bar(['Giá TB'], [float(basic_stats['avg_price'].replace(',', ''))], color='lightgreen')
    ax2.set_title('Giá trung bình', fontsize=14, fontweight='bold')
    ax2.set_ylabel('VNĐ')
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))
    ax2.grid(axis='y', alpha=0.3)

    # Diện tích trung bình
    ax3.bar(['Diện tích TB'], [basic_stats['avg_area']], color='lightcoral')
    ax3.set_title('Diện tích trung bình', fontsize=14, fontweight='bold')
    ax3.set_ylabel('m²')
    ax3.grid(axis='y', alpha=0.3)

    # Số thành phố
    ax4.bar(['Số thành phố'], [basic_stats['cities_count']], color='gold')
    ax4.set_title('Số thành phố', fontsize=14, fontweight='bold')
    ax4.set_ylabel('Số lượng')
    ax4.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Hiển thị bảng thống kê
    print("📊 Thống kê cơ bản:")
    display(basic_stats.to_frame().T)

🕓 DESCRIBE HISTORY (Lịch sử thay đổi bảng):
Version 5: 1762940920468 - MERGE
Version 4: 1762866568088 - MERGE
Version 3: 1762866070414 - MERGE
Version 2: 1762865823689 - MERGE
Version 1: 1762865525717 - MERGE
Version 0: 1762865074576 - WRITE



In [ ]:
# ============================================================
# 2. BIỂU ĐỒ GIÁ THEO THÀNH PHỐ
# ============================================================

if 'price_by_city' in results and not results['price_by_city'].empty:
    price_city = results['price_by_city'].head(10).copy()

    # Chuyển đổi giá sang số để vẽ biểu đồ
    price_city['avg_price_num'] = price_city['avg_price'].str.replace(',', '').astype(float)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # Biểu đồ cột giá trung bình
    bars = ax1.bar(range(len(price_city)), price_city['avg_price_num'], color='steelblue', alpha=0.8)
    ax1.set_title('Giá trung bình bất động sản theo thành phố (Top 10)', fontsize=16, fontweight='bold')
    ax1.set_xlabel('Thành phố', fontsize=12)
    ax1.set_ylabel('Giá trung bình (VNĐ)', fontsize=12)
    ax1.set_xticks(range(len(price_city)))
    ax1.set_xticklabels([city[:30] + '...' if len(city) > 30 else city for city in price_city['city']],
                       rotation=45, ha='right', fontsize=10)
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e9:.1f} tỷ'))
    ax1.grid(axis='y', alpha=0.3)

    # Thêm giá trị trên cột
    for bar, price in zip(bars, price_city['avg_price_num']):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{price/1e9:.1f} tỷ', ha='center', va='bottom', fontsize=9, fontweight='bold')

    # Biểu đồ số lượng bất động sản
    ax2.bar(range(len(price_city)), price_city['property_count'], color='darkorange', alpha=0.8)
    ax2.set_title('Số lượng bất động sản theo thành phố (Top 10)', fontsize=16, fontweight='bold')
    ax2.set_xlabel('Thành phố', fontsize=12)
    ax2.set_ylabel('Số lượng', fontsize=12)
    ax2.set_xticks(range(len(price_city)))
    ax2.set_xticklabels([city[:30] + '...' if len(city) > 30 else city for city in price_city['city']],
                       rotation=45, ha='right', fontsize=10)
    ax2.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Hiển thị bảng dữ liệu
    print("📊 Giá theo thành phố (Top 10):")
    display(price_city[['city', 'property_count', 'avg_price', 'min_price', 'max_price']])

📦 Đang đọc dữ liệu từ MinIO (DuckDB)...
✅ Đọc thành công 142 dòng dữ liệu.
📋 Các cột có sẵn:
['url', 'Diện tích', 'Mức giá', 'Địa chỉ', 'Ngày đăng', 'Tiêu đề', 'latitude', 'longitude', 'search_city', 'search_property_type', 'search_rent_or_buy', 'search_radius', 'propertyDetails_propertyId']

📊 Thống kê mô tả:
                            count unique  \
url                           142     36   
Diện tích                     142     18   
Mức giá                       142     25   
Địa chỉ                       142     31   
Ngày đăng                     142      2   
Tiêu đề                       142     36   
latitude                    142.0    NaN   
longitude                   142.0    NaN   
search_city                   142      4   
search_property_type          142      1   
search_rent_or_buy            142      1   
search_radius               142.0    NaN   
propertyDetails_propertyId    142     36   

                                                                       

In [ ]:
# ============================================================
# 3. BIỂU ĐỒ PHÂN BỐ THEO KHU VỰC
# ============================================================

if 'location_distribution' in results and not results['location_distribution'].empty:
    location_dist = results['location_distribution'].head(15).copy()

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # Biểu đồ tròn phần trăm
    wedges, texts, autotexts = ax1.pie(location_dist['percentage'], labels=None, autopct='%1.1f%%',
                                      startangle=90, colors=plt.cm.Set3.colors)
    ax1.set_title('Phân bố bất động sản theo khu vực (%)', fontsize=16, fontweight='bold')
    ax1.axis('equal')

    # Legend riêng
    ax1.legend(wedges, [f'{loc[:40]}...' if len(loc) > 40 else loc for loc in location_dist['location']],
              title="Khu vực", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1), fontsize=10)

    # Biểu đồ cột số lượng
    bars = ax2.bar(range(len(location_dist)), location_dist['count'], color='teal', alpha=0.7)
    ax2.set_title('Số lượng bất động sản theo khu vực', fontsize=16, fontweight='bold')
    ax2.set_xlabel('Khu vực', fontsize=12)
    ax2.set_ylabel('Số lượng', fontsize=12)
    ax2.set_xticks(range(len(location_dist)))
    ax2.set_xticklabels([loc[:25] + '...' if len(loc) > 25 else loc for loc in location_dist['location']],
                       rotation=45, ha='right', fontsize=10)
    ax2.grid(axis='y', alpha=0.3)

    # Thêm giá trị trên cột
    for bar, count in zip(bars, location_dist['count']):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{int(count)}', ha='center', va='bottom', fontsize=9, fontweight='bold')

    plt.tight_layout()
    plt.show()

    # Hiển thị bảng dữ liệu
    print("📊 Phân bố theo khu vực:")
    display(location_dist)

📈 SELECT COUNT(*) FROM delta:
   total_rows
0         142

📋 DESCRIBE (Cấu trúc cơ bản):
┌────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│        column_name         │ column_type │  null   │   key   │ default │  extra  │
│          varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ url                        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Diện tích                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Mức giá                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Địa chỉ                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Ngày đăng                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Tiêu đề                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ latitude                   │ DOUBLE      │ YES     │ NULL  

In [ ]:
# ============================================================
# 4. BIỂU ĐỒ PHÂN BỐ THEO KHOẢNG GIÁ
# ============================================================

if 'price_ranges' in results and not results['price_ranges'].empty:
    price_ranges = results['price_ranges'].copy()

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # Biểu đồ cột số lượng theo khoảng giá
    bars = ax1.bar(range(len(price_ranges)), price_ranges['count'], color='crimson', alpha=0.8)
    ax1.set_title('Phân bố bất động sản theo khoảng giá', fontsize=16, fontweight='bold')
    ax1.set_xlabel('Khoảng giá', fontsize=12)
    ax1.set_ylabel('Số lượng bất động sản', fontsize=12)
    ax1.set_xticks(range(len(price_ranges)))
    ax1.set_xticklabels(price_ranges['price_range'], rotation=45, ha='right', fontsize=11)
    ax1.grid(axis='y', alpha=0.3)

    # Thêm giá trị trên cột
    for bar, count in zip(bars, price_ranges['count']):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{int(count)}', ha='center', va='bottom', fontsize=10, fontweight='bold')

    # Biểu đồ tròn phần trăm
    wedges, texts, autotexts = ax2.pie(price_ranges['percentage'], labels=price_ranges['price_range'],
                                      autopct='%1.1f%%', startangle=90, colors=plt.cm.Pastel1.colors)
    ax2.set_title('Tỷ lệ phần trăm theo khoảng giá', fontsize=16, fontweight='bold')
    ax2.axis('equal')

    plt.tight_layout()
    plt.show()

    # Hiển thị bảng dữ liệu
    print("📊 Phân bố theo khoảng giá:")
    display(price_ranges)

# ============================================================
# 5. BIỂU ĐỒ XU HƯỚNG THỜI GIAN (30 NGÀY GẦN NHẤT)
# ============================================================

if 'recent_trends' in results and not results['recent_trends'].empty:
    trends = results['recent_trends'].copy()

    # Chuyển đổi date sang datetime
    trends['date'] = pd.to_datetime(trends['date'])
    trends = trends.sort_values('date')

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 12))

    # Xu hướng số lượng hàng ngày
    ax1.plot(trends['date'], trends['daily_count'], marker='o', linewidth=2, markersize=4, color='navy')
    ax1.set_title('Xu hướng số lượng bất động sản hàng ngày', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Ngày', fontsize=12)
    ax1.set_ylabel('Số lượng', fontsize=12)
    ax1.grid(True, alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)

    # Xu hướng giá trung bình
    ax2.plot(trends['date'], trends['avg_price'].str.replace(',', '').astype(float),
             marker='s', linewidth=2, markersize=4, color='darkgreen')
    ax2.set_title('Xu hướng giá trung bình hàng ngày', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Ngày', fontsize=12)
    ax2.set_ylabel('Giá trung bình (VNĐ)', fontsize=12)
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e9:.1f} tỷ'))
    ax2.grid(True, alpha=0.3)
    ax2.tick_params(axis='x', rotation=45)

    # Xu hướng diện tích trung bình
    ax3.plot(trends['date'], trends['avg_area'], marker='^', linewidth=2, markersize=4, color='darkorange')
    ax3.set_title('Xu hướng diện tích trung bình hàng ngày', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Ngày', fontsize=12)
    ax3.set_ylabel('Diện tích trung bình (m²)', fontsize=12)
    ax3.grid(True, alpha=0.3)
    ax3.tick_params(axis='x', rotation=45)

    # Scatter plot giá vs diện tích
    ax4.scatter(trends['avg_area'], trends['avg_price'].str.replace(',', '').astype(float),
               s=trends['daily_count']*10, alpha=0.6, color='purple', edgecolors='black')
    ax4.set_title('Mối quan hệ giá và diện tích theo ngày', fontsize=14, fontweight='bold')
    ax4.set_xlabel('Diện tích trung bình (m²)', fontsize=12)
    ax4.set_ylabel('Giá trung bình (VNĐ)', fontsize=12)
    ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e9:.1f} tỷ'))
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Hiển thị bảng xu hướng
    print("📊 Xu hướng 30 ngày gần nhất:")
    display(trends)

✅ Đã xuất thành công file TEST.pdf
